In [12]:
print(getwd())


[1] "c:/Users/kenne/Documents/GitHub/lecospec/Functions"


In [18]:
source("./LandCoverEstimator_RF.R")

In [17]:
raster_data_path <- "F:/DataCubes/raw_1511_rd_rf_or"
output_csv_filepath <- "E:/Lecospec/raw_2000_converted.csv"
output_parquet_filepath <- "../Data/raw_2000_converted.parquet"
tile_path <- "F:/DataCubes/1511_tiles/"
config_path <- "./config.json"


In [19]:
results <- estimate_land_cover(raster_data_path)

[1] "8 Cores Detected for processing..."


ERROR: Error in {: task 1 failed - "argument "model" is missing, with no default"


In [3]:
test_raster <- raster::brick(raster_data_path)


In [8]:
model <- load("../Output/E_004_Best_Model_Ranger.rda")
print(model)

[1] "rf_mod_ranger"


In [1]:
av <- c("NDVI","OSAVI","SAVI","MTVI","NDWI","PWI",
          "MSI", "SRWI","GMI1","GMI2","MCARI","TVI",
          "Vogelmann4","Boochs","Boochs2",
          "CARI","CI","Carter","Carter2","Carter3","Carter4",
          "Carter5","Carter6","Datt","Datt2","Datt3","Datt4",
          "Datt5","Datt6","DD","DDn","D1","D2","EVI","EGFR","EGFN",
          "GI","Gitelson","Gitelson2","Green NDVI","MCARI/OSAVI",
          "MCARI2","MCARI2/OSAVI2","mNDVI","mND705","Maccioni",
          "mREIP","MSAVI","mSR","mSR705","mSR2","MTCI","NDVI2",
          "NDVI3","NPCI","OSAVI2","RDVI","REP_LE","REP_Li",
          "SIPI","SPVI","SR","SR1","SR2","SR3","SR4","SR5","SR6",
          "SR7", "SR8","SRPI","Sum_Dr1","Sum_Dr2","TCARI","TCARI2",
          "TCARI/OSAVI","TCARI2/OSAVI2","Vogelmann","NDLI",
          "Vogelmann2","Vogelmann3","PRI","CAI","NDNI",
          "PSSR", "PSND", "CRI1", "CRI2", "CRI3",
          "CRI4", "MPRI", "PRI*CI2", "CI2", "PSRI", "ClAInt", 
          "TGI", "PRI_norm","PARS","DPI","Datt7","Datt8",
          "GDVI_2","GDVI_3","GDVI_4","LWVI1","LWVI2",
          "DWSI1","DWSI2","DWSI3","DWSI4","DWSI5",
          "SWIR FI", "SWIR LI", "SWIR SI", "SWIR VI"
          )
          

In [4]:
base_index <- hsdar::vegindex()
AVIRIS_VI  <-base_index [-58]
headwall_VI<-base_index [-c(3,26,27,31,32,33,35,48,49,58,60,66,67,71,82,99,102,103,104,105)]
print(headwall_VI)
write.csv(headwall_VI, "../vegIndicesUsed.csv")

 [1] "Boochs"        "Boochs2"       "CARI"          "Carter"       
 [5] "Carter2"       "Carter3"       "Carter4"       "Carter5"      
 [9] "Carter6"       "CI"            "CI2"           "ClAInt"       
[13] "CRI1"          "CRI2"          "CRI3"          "CRI4"         
[17] "D1"            "D2"            "Datt"          "Datt2"        
[21] "Datt3"         "Datt4"         "Datt5"         "Datt6"        
[25] "DD"            "DDn"           "DPI"           "DWSI4"        
[29] "EGFN"          "EGFR"          "EVI"           "GDVI_2"       
[33] "GDVI_3"        "GDVI_4"        "GI"            "Gitelson"     
[37] "Gitelson2"     "GMI1"          "GMI2"          "Green NDVI"   
[41] "Maccioni"      "MCARI"         "MCARI/OSAVI"   "MCARI2"       
[45] "MCARI2/OSAVI2" "mND705"        "mNDVI"         "MPRI"         
[49] "MSAVI"         "mSR"           "mSR2"          "mSR705"       
[53] "MTCI"          "MTVI"          "NDVI"          "NDVI2"        
[57] "NDVI3"         "NPCI"       

In [5]:
d_indexs <- c("Boochs","Boochs2","Datt3","D1","D2","EGFR","EGFN",
              "Vogelmann3","Sum_Dr1","Sum_Dr2","REP_LE","DPI")

In [6]:
intersect(d_indexs, headwall_VI)

[1] "Boochs"     "Boochs2"    "Datt3"      "D1"         "D2"        
 [6] "EGFR"       "EGFN"       "Vogelmann3" "Sum_Dr1"    "Sum_Dr2"   
[11] "REP_LE"     "DPI"

In [4]:
library(parallel)
library(raster)
library(SpaDES)

Warning message:
"package 'raster' was built under R version 4.1.1"
Loading required package: sp

Warning message:
"package 'sp' was built under R version 4.1.1"

Attaching package: 'raster'


The following object is masked from 'package:dplyr':

    select


Warning message:
"package 'SpaDES' was built under R version 4.1.1"
Warning message:
"package 'reproducible' was built under R version 4.1.1"
Warning message:
"package 'quickPlot' was built under R version 4.1.1"
Warning message:
"package 'SpaDES.core' was built under R version 4.1.1"
Warning message:
"package 'SpaDES.tools' was built under R version 4.1.1"
loading reproducible    1.2.8
loading quickPlot       0.1.6
loading SpaDES.core     1.0.9
loading SpaDES.tools    0.3.9


Default paths for SpaDES directories set to:
  cachePath:  
  inputPath:  D:\temp\RtmpCAt7QD/SpaDES/inputs
  modulePath: D:\temp\RtmpCAt7QD/SpaDES/modules
  outputPath: D:\temp\RtmpCAt7QD/SpaDES/outputs
These can be changed using 'setPaths()'. See '?setPaths

In [7]:
cl = beginCluster()

test_tile_filenames <- make_tiles(
                            test_raster,
                            save_path = tile_path,
                            cluster = cl,
                            verbose = TRUE
                        )

endCluster()



8 cores detected, using 7



[1] "Creating tile templates"
[1] "Templates created, cropping raster"
[1] "complete"


In [8]:
rm(test_raster)
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,3445448,184.1,6796524,363.0,4480012,239.3
Vcells,5420234,41.4,23041553,175.8,45145416,344.5


In [11]:
print(as.character(extent(test_raster)))

[1] "extent(-148.95288965, -148.949812003, 63.806721752, 63.807439657)"


In [14]:
install.packages("stringi")

Installing package into 'C:/Users/kenne/Documents/R/win-library/4.1'
(as 'lib' is unspecified)



package 'stringi' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'stringi'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\kenne\Documents\R\win-library\4.1\00LOCK\stringi\libs\icudt69l.dat to C:\Users\kenne\Documents\R\win-library\4.1\stringi\libs\icudt69l.dat: Invalid argument"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\kenne\Documents\R\win-library\4.1\00LOCK\stringi\libs\x64\stringi.dll to C:\Users\kenne\Documents\R\win-library\4.1\stringi\libs\x64\stringi.dll: Permission denied"
Warning message:
"restored 'stringi'"



The downloaded binary packages are in
	C:\Users\kenne\AppData\Local\Temp\RtmpWqnj7v\downloaded_packages


## Data Cubes for Tree Use Case
